### Using Random Forest to predict rainfall

### What is our Objective ?
* To reliably predict next day's rainfall using possible derminants.

### Getting to know our WeatherAus dataset! 

WeatherAus dataset contains about 10 years of daily weather observations from many locations across Australia it has the following features:

* Date - The date of observation.
* Location - Location of the weather station.
* MinTemp - Minimum temperature of the observed day in degree celsius.
* MaxTemp - Maximum temperature of the observed day in degree celsius.
* Rainfall - The amount of rainfall recorded for the day in mm.
* Evaporation - Class A pan evaporation (mm) in the 24 hours to 9am.
* Sunshine - The number of hours of bright sunshine in the day.
* WindGustDir - The direction of the strongest wind gust in the 24 hours to midnight.
* WindGustSpeed - The speed (km/h) of the strongest wind gust in the 24 hours to midnight.
* WindDir9am - Direction of the wind at 9am.
* WindDir3pm - Direction of the wind at 3pm.
* WindSpeed9am - Wind speed (km/hr) averaged over 10 minutes prior to 9am.
* WindSpeed3pm - Wind speed (km/hr) averaged over 10 minutes prior to 3pm.
* Humidity9am - Humidity (percent) at 9am.
* Humidity3pm - Humidity (percent) at 3pm.
* Pressure9am - Atmospheric pressure (hpa) reduced to mean sea level at 9am.
* Pressure3pm - Atmospheric pressure (hpa) reduced to mean sea level at 3pm.
* Cloud9am - Fraction of sky obscured by cloud at 9am. This is measured in "oktas", which are a unit of eigths. It records how many.
* Cloud3pm - Fraction of sky obscured by cloud at 3pm. This is measured in "oktas", which are a unit of eigths. It records how many.
* Temp9am - Temperature (degrees C) at 9am.
* Temp3pm - Temperature (degrees C) at 3pm.
* RainToday -  if precipitation (mm) in the 24 hours to 9am exceeds 1mm, otherwise 0.
* RainTomorrow - The amount of next day rain in mm.

### Approach
* In this example, we will be balancing an imbalanced data set using random oversampling, undersampling and SMOTE.
* Then we'll label encode categorical features.
* Use various imputation methods to handle missing values in the dataset.
* Split the preprocessed dataset and train on it using RandomForest API from mlpack.
* Finally we'll use various metrics such as Accuracy, F1-Score, ROC AUC to judge the performance of our model.

#### NOTE: This example has 4 parts implementing the above approach using raw imbalanced data, undersampled, oversampled & using SMOTE. 

In [1]:
!wget -q http://datasets.mlpack.org/weatherAUS.csv

In [2]:
// Import necessary library headers.
#include <mlpack/xeus-cling.hpp>
#include <mlpack.hpp>

In [3]:
// Import utility headers.
#define WITHOUT_NUMPY 1
#include "matplotlibcpp.h"
#include "xwidgets/ximage.hpp"
#include "../utils/preprocess.hpp"
#include "../utils/plot.hpp"

namespace plt = matplotlibcpp;

In [4]:
using namespace mlpack;

In [5]:
using namespace mlpack::data;

### Part 1 - Modelling using Imbalanced Dataset

### Visualize the Missing Values

In [6]:
// Generate a heatmap for missing values.
MissingPlot("weatherAUS.csv", "PuBu", "Part-1 Missing values pre-imputation");
auto img = xw::image_from_file("./plots/Part-1 Missing values pre-imputation.png").finalize();
img

A Jupyter widget with unique id: b6a1930ac1e749818a7bd810bc74feeb

The above visualization shows that high number of missing values in: Sunshine, Evaporation, Cloud9am and Cloud3pm.
We observe that atmost some features have 50% missing values. So instead of discarding them, we will impute them with  proper imputation method.

In [7]:
// Perform imputation on the original dataset using "mean" imputation policy.
Impute("weatherAUS.csv");

Drop the dataset header using sed, sed is a Unix utility that parses and transforms text.

In [8]:
!cat ./data/weatherAUS_mean_imputed.csv | sed 1d > ./data/weatherAUS_trim.csv

Drop columns 1 ("Date") as it is not required and causes issues while loading the data.

In [9]:
!cut -d, -f1 --complement ./data/weatherAUS_trim.csv > ./data/weatherAUS_trim2.csv

Rename the newly created csv file.

In [10]:
!rm ./data/weatherAUS_trim.csv

In [11]:
!mv ./data/weatherAUS_trim2.csv ./data/weatherAUS_trim.csv

In [12]:
arma::mat LoadData(const std::string fname)
{
    arma::mat data;
    mlpack::data::DatasetInfo info;

    // Manually set the columns with contain categorical data in DatasetInfo.
    info.Type(0) = mlpack::data::Datatype::categorical;
    info.Type(6) = mlpack::data::Datatype::categorical;
    info.Type(8) = mlpack::data::Datatype::categorical;
    info.Type(9) = mlpack::data::Datatype::categorical;
    info.Type(20) = mlpack::data::Datatype::categorical;
    info.Type(21) = mlpack::data::Datatype::categorical;

    data::Load(fname.c_str(), data, info);
    
    return data;
}

In [13]:
// Load the preprocessed dataset into armadillo matrix.
arma::mat weatherData = LoadData("./data/weatherAUS_trim.csv")

In [14]:
// Save the label encoded data for visualization.
data::Save("./data/weatherAUSEnc.csv", weatherData);

In [15]:
!sed -i '1iLocation,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow' ./data/weatherAUSEnc.csv

In [16]:
// Inspect the first 5 examples in the dataset.
std::cout.precision(4);
std::cout.setf(std::ios::fixed);
std::cout << std::setw(15) << "Location" << std::setw(10) << "MinTemp" << std::setw(13) << "MaxTemp" 
          << std::setw(12) << "Rainfall" << std::setw(15) << "Evaporation" << std::setw(12) 
          << "Sunshine" << std::setw(14) << "WindGust" << std::setw(15) << "WindGustSpeed"
          << std::setw(12) << "WindDir9am" << std::setw(12) << "WindDir3pm" << std::setw(13)
          << "WindSpeed9am" << std::setw(14) << "WindSpeed3pm" << std::setw(13) 
          << "Humidity9am" << std::setw(12) << "Humidity3pm" << std::setw(14)
          << "Pressure9am" << std::setw(14) << "Pressure3pm" << std::setw(10) 
          << "Cloud9am" << std::setw(14) << "Cloud3pm" << std::setw(15)
          << "Temp9am" << std::setw(12) << "Temp3pm" << std::setw(16)
          << "RainToday" << std::setw(15) << "RainTomorrow" << std::endl;
std::cout << weatherData.submat(0, 0, weatherData.n_rows-1, 5).t() << std::endl;

       Location   MinTemp      MaxTemp    Rainfall    Evaporation    Sunshine      WindGust  WindGustSpeed  WindDir9am  WindDir3pm WindSpeed9am  WindSpeed3pm  Humidity9am Humidity3pm   Pressure9am   Pressure3pm  Cloud9am      Cloud3pm        Temp9am     Temp3pm       RainToday   RainTomorrow
            0   1.3400e+01   2.2900e+01   6.0000e-01   5.4682e+00   7.6112e+00            0   4.4000e+01            0            0   2.0000e+01   2.4000e+01   7.1000e+01   2.2000e+01   1.0077e+03   1.0071e+03   8.0000e+00   4.5099e+00   1.6900e+01   2.1800e+01            0            0
            0   7.4000e+00   2.5100e+01            0   5.4682e+00   7.6112e+00   1.0000e+00   4.4000e+01   1.0000e+00   1.0000e+00   4.0000e+00   2.2000e+01   4.4000e+01   2.5000e+01   1.0106e+03   1.0078e+03   4.4475e+00   4.5099e+00   1.7200e+01   2.4300e+01            0            0
            0   1.2900e+01   2.5700e+01            0   5.4682e+00   7.6112e+00   2.0000e+00   4.6000e+01            0   1.0000e+00   

In [17]:
// Visualize the distribution of target classes.
CountPlot("./data/weatherAUS_mean_imputed.csv", "RainTomorrow", "", "Part-1 Distribution of target class");
auto img = xw::image_from_file("./plots/Part-1 Distribution of target class.png").finalize();
img

A Jupyter widget with unique id: bb6f6c4a480f4c3b96343746096d2b70

From the above visualization, we can observe that the presence of "No" and "Yes" are in ratio 78:22, so there is a huge class imbalance. For the first part we would not be handling the class imbalance. In order to see how our model performs on the raw imbalanced data.

### EDA

In [18]:
// Visualize the direction of wind at 9 AM.
CountPlot("./data/weatherAUS_mean_imputed.csv", "WindDir9am", "", "Part-1 Direction of wind at 9 am");
auto img = xw::image_from_file("./plots/Part-1 Direction of wind at 9 am.png").finalize();
img

A Jupyter widget with unique id: b57c8b1475ae4d20ba46f18b666373de

In [19]:
// Visualize the direction of wind at 3 PM.
CountPlot("./data/weatherAUS_mean_imputed.csv", "WindDir3pm", "", "Part-1 Direction of wind at 3 pm");
auto img = xw::image_from_file("./plots/Part-1 Direction of wind at 3 pm.png").finalize();
img

A Jupyter widget with unique id: dce79b0d88c14614acd15b0334a90974

In [20]:
// Visualize the direction of wind gust in different directions.
CountPlot("./data/weatherAUS_mean_imputed.csv", "WindGustDir", "", "Part-1 Direction of wind Gust");
auto img = xw::image_from_file("./plots/Part-1 Direction of wind Gust.png").finalize();
img

A Jupyter widget with unique id: b54edbc10f5a44d9a61192a037a99426

### Visualize Correlation

In [21]:
// Visualize the correlation of transformed dataset.
HeatMapPlot("./data/weatherAUSEnc.csv", "coolwarm", "Part-1 Correlation Heatmap", 1);
auto img = xw::image_from_file("./plots/Part-1 Correlation Heatmap.png").finalize();
img

A Jupyter widget with unique id: 14bcf4439a4743a1bb24166fc557d763

As we can observe from the above heatmap, there is high correlation between the following features:
* MinTemp & MaxTemp.
* MinTemp & Temp9am.
* MinTemp & Temp3pm.
* MaxTemp & Temp9am.
* MaxTemp & Temp3pm.
* Temp3pm & Temp9am.
* Pressure9am & Pressure3pm.
* Evaporation & MaxTemp.

In [22]:
// Split the data into features (X) and target (y) variables, targets are the last row.
arma::Row<size_t> targets = arma::conv_to<arma::Row<size_t>>::from(weatherData.row(weatherData.n_rows - 1));
// Targets are dropped from the loaded matrix.
weatherData.shed_row(weatherData.n_rows-1);

### Train Test Split

The dataset has to be split into training and test set. Here the dataset has 145460 observations and the test ratio is taken as 25% of the total observations. This indicates that the test set should have 25% * 145460 = 36365 observations and training set should have 109095 observations respectively.

In [23]:
// Split the dataset into train and set sets using mlpack Split API.
arma::mat Xtrain, Xtest;
arma::Row<size_t> Ytrain, Ytest;
Split(weatherData, targets, Xtrain, Xtest, Ytrain, Ytest, 0.25);

In [24]:
// Standardize the train & test features.
arma::mat XtrainScaled, XtestScaled;
StandardScaler scale;
scale.Fit(Xtrain);
scale.Transform(Xtrain, XtrainScaled);
scale.Transform(Xtest, XtestScaled);

### Training the Random Forest model
Random forest is a commonly-used machine learning algorithm, which combines the output of multiple decision trees to reach a single result. Random Forest algorithm is an extension of the bagging method as it utilizes both bagging and feature randomness to create an uncorrelated forest of decision trees. While decision trees consider all the possible feature splits, random forests only select a subset of those features.
To create the model we'll be using `RandomForest` from mlpack.
We use only 5 trees for the sake of preserving computational resources, but better models could be obtained with more trees.

In [25]:
//Create and train Random Forest model with 5 trees.
RandomForest<> rf(XtrainScaled, Ytrain, 2, 5);

### Making Predictions on Test set

In [26]:
// Predict the values for test data using previously trained model as input.
arma::Row<size_t> output;
arma::mat probs;
rf.Classify(XtestScaled, output, probs);

In [27]:
// Save predicted probabilities and ground truth as csv for generating ROC AUC curve.
data::Save("./data/probabilities.csv", probs);
data::Save("./data/ytest.csv", Ytest);

In [28]:
double ComputeAccuracy(const arma::Row<size_t>& yPreds, const arma::Row<size_t>& yTrue)
{
    const size_t correct = arma::accu(yPreds == yTrue);
    return (double)correct / (double)yTrue.n_elem;
}

In [29]:
double ComputePrecision(const size_t truePos, const size_t falsePos)
{
    return (double)truePos / (double)(truePos + falsePos);
}

In [30]:
double ComputeRecall(const size_t truePos, const size_t falseNeg)
{
    return (double)truePos / (double)(truePos + falseNeg);
}

In [31]:
double ComputeF1Score(const size_t truePos, const size_t falsePos, const size_t falseNeg)
{
    double prec = ComputePrecision(truePos, falsePos);
    double rec = ComputeRecall(truePos, falseNeg);
    return 2 * (prec * rec) / (prec + rec);
}

In [32]:
void ClassificationReport(const arma::Row<size_t>& yPreds, const arma::Row<size_t>& yTrue)
{
    arma::Row<size_t> uniqs = arma::unique(yTrue);
    std::cout << std::setw(29) << "precision" << std::setw(15) << "recall" 
              << std::setw(15) << "f1-score" << std::setw(15) << "support" 
              << std::endl << std::endl;
    
    for(auto val: uniqs)
    {
        size_t truePos = arma::accu(yTrue == val && yPreds == val && yPreds == yTrue);
        size_t falsePos = arma::accu(yPreds == val && yPreds != yTrue);
        size_t trueNeg = arma::accu(yTrue != val && yPreds != val && yPreds == yTrue);
        size_t falseNeg = arma::accu(yPreds != val && yPreds != yTrue);
        
        std::cout << std::setw(15) << val
                  << std::setw(12) << std::setprecision(2) << ComputePrecision(truePos, falsePos) 
                  << std::setw(16) << std::setprecision(2) << ComputeRecall(truePos, falseNeg) 
                  << std::setw(14) << std::setprecision(2) << ComputeF1Score(truePos, falsePos, falseNeg)
                  << std::setw(16) << truePos
                  << std::endl;
    }
}

### Evaluation metrics

* True Positive - The actual value was true & the model predicted true.
* False Positive - The actual value was false & the model predicted true, Type I error.
* True Negative - The actual value was false & the model predicted false.
* False Negative - The actual value was true & the model predicted false, Type II error.

`Accuracy`: is a metric that generally describes how the model performs across all classes. It is useful when all classes are of equal importance. It is calculated as the ratio between the number of correct predictions to the total number of predictions.

$$Accuracy = \frac{True_{positive} + True_{negative}}{True_{positive} + True_{negative} + False_{positive} + False_{negative}}$$

`Precision`: is calculated as the ratio between the number of positive samples correctly classified to the total number of samples classified as Positive. The precision measures the model's accuracy in classifying a sample as positive.

$$Precision = \frac{True_{positive}}{True_{positive} + False_{positive}}$$

`Recall`: is calulated as the ratio between the number of positive samples correctly classified as Positive to the total number of Positive samples. The recall measures the model's ability to detect Positive samples. The higher the recall, the more positive samples detected.

$$Recall = \frac{True_{positive}}{True_{positive} + False_{negative}}$$

* The decision of whether to use precision or recall depends on the type of problem begin solved.
* If the goal is to detect all positive samples then use recall.
* Use precision if the problem is sensitive to classifying a sample as Positive in general.

* ROC graph has the True Positive rate on the y axis and the False Positive rate on the x axis.
* ROC Area under the curve in the graph is the primary metric to determine if the classifier is doing well, the higher the value the higher the model performance.

### Model Evaluation

In [33]:
// Classification report.
std::cout << "Accuracy: " << ComputeAccuracy(output, Ytest) << std::endl;
ClassificationReport(output, Ytest);

Accuracy: 0.8326
                    precision         recall       f1-score        support

              0        0.87            0.93          0.90           26481
              1        0.66            0.48          0.55            3795


In [34]:
// Plot ROC AUC Curve to visualize the performance of the model on TP & FP.
RocAucPlot("./data/ytest.csv", "./data/probabilities.csv", "Part-1 Imbalanced Targets ROC AUC Curve");
auto img = xw::image_from_file("./plots/Part-1 Imbalanced Targets ROC AUC Curve.png").finalize();
img

A Jupyter widget with unique id: 3eede6779b5449b8b8c8ffd6890c4378

From the above classification report, we can infer that our model trained on imbalanced data performs well on negative class but not the same for positive class.
Also from the ROC AUC Curve, we can infer the same.

### Part 2 - Modelling using Random Oversampling

For this part we would be handling the class imbalance. In order to see how our model performs on the randomly oversampled data. We will be using `Resample()` method to oversample the minority class i.e "Yes, signifying it will rain tomorrow".

In [35]:
// Oversample the minority population.
Resample("weatherAUS.csv", "RainTomorrow", "No", "Yes", "oversample", "Date", 123);

In [36]:
// Visualize the distribution of target classes.
CountPlot("./data/weatherAUS_oversampled.csv", "RainTomorrow", "", "Part-2 Oversampled Population");
auto img = xw::image_from_file("./plots/Part-2 Oversampled Population.png").finalize();
img

A Jupyter widget with unique id: 000f9bc21b844691a524847d111602a5

From the above plot we can see that after resampling the minority class (Yes) is oversampled to be equal to the majority class (No). This solves our imbalanced data issue for this part.

### Visualize the Missing Values

In [37]:
// Generate a heatmap for missing values.
MissingPlot("./data/weatherAUS_oversampled.csv", "PuBu", "Part-2 Missing values before imputation");
auto img = xw::image_from_file("./plots/Part-2 Missing values before imputation.png").finalize();
img

A Jupyter widget with unique id: 477711f77af84e57862866e629399ab3

The above visualization shows that high number of missing values in: Sunshine, Evaporation, Cloud9am and Cloud3pm.
We observe that atmost some features have 50% missing values. So instead of discarding them, we will impute them with  proper imputation method.

In [38]:
// Imputation using mean strategy.
Impute("./data/weatherAUS_oversampled.csv");

In [39]:
!cat ./data/weatherAUS_oversampled_mean_imputed.csv | sed 1d > ./data/weatherAUS_os_imp.csv

In [40]:
!cut -d, -f1 --complement ./data/weatherAUS_os_imp.csv > ./data/weatherAUS_trim2.csv

In [41]:
!rm ./data/weatherAUS_trim.csv

In [42]:
!mv ./data/weatherAUS_trim2.csv ./data/weatherAUS_trim.csv

In [43]:
// Load the oversampled encoded & trimmed data into armadillo matrix.
arma::mat overSampled = LoadData("./data/weatherAUS_trim.csv")

In [44]:
// Save the encoded armadillo matrix for visualization.
data::Save("./data/weatherAUSEnc.csv", overSampled);

In [45]:
!sed -i '1iLocation,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow' ./data/weatherAUSEnc.csv

### Visualize Correlation

In [46]:
// Plot the correlation matrix as heatmap.
HeatMapPlot("./data/weatherAUSEnc.csv", "coolwarm", "Part-2 Correlation Heatmap", 1);
auto img = xw::image_from_file("./plots/Part-2 Correlation Heatmap.png").finalize();
img

A Jupyter widget with unique id: 88bb2a840bae4f47a9c094370863405a

In [47]:
// Split the data into features (X) and target (y) variables, targets are the last row.
arma::Row<size_t> targets = arma::conv_to<arma::Row<size_t>>::from(overSampled.row(overSampled.n_rows - 1));
// Targets are dropped from the loaded matrix.
overSampled.shed_row(overSampled.n_rows-1);

### Train Test Split
The dataset has to be split into training and test set. Here the dataset has 220632 observations and the test ratio is taken as 25% of the total observations. This indicates that the test set should have 25% * 220632 = 55158 observations and training set should have 165474 observations respectively.

In [48]:
// Split the dataset into train and set sets using mlpack Split API.
Split(overSampled, targets, Xtrain, Xtest, Ytrain, Ytest, 0.25);

In [49]:
// Standardize the train & test features.
StandardScaler scale;
scale.Fit(Xtrain);
scale.Transform(Xtrain, XtrainScaled);
scale.Transform(Xtest, XtestScaled);

### Training the Random Forest model
We will use `RandomForest` from mlpack to train the model with 5 trees on the oversampled data.
We use only 5 trees for the sake of preserving computational resources, but better models could be obtained with more trees.

In [50]:
// Create and train Random Forest model with 5 trees.
rf = RandomForest<>(XtrainScaled, Ytrain, 2, 5);

### Making Predictions on Test set

In [51]:
// Predict the values for test data using previously trained model as input.
rf.Classify(XtestScaled, output, probs);

In [52]:
// Save predicted probabilities and ground truth as csv for generating ROC AUC curve.
data::Save("./data/probabilities.csv", probs);
data::Save("./data/ytest.csv", Ytest);

### Model Evaluation

In [53]:
// Classification report.
std::cout << "Accuracy: " << ComputeAccuracy(output, Ytest) << std::endl;
ClassificationReport(output, Ytest);

Accuracy: 0.92
                    precision         recall       f1-score        support

              0        0.95            0.88          0.91           24304
              1        0.89            0.95          0.92           26240


In [54]:
// Plot ROC AUC Curve to visualize the performance of the model on TP & FP.
RocAucPlot("./data/ytest.csv", "./data/probabilities.csv", "Part-2 Random Oversampled Targets ROC AUC Curve");
auto img = xw::image_from_file("./plots/Part-2 Random Oversampled Targets ROC AUC Curve.png").finalize();
img

A Jupyter widget with unique id: c53fab662a3d4b1c9606c80622eeaa05

### Part 3 - Modelling using Synthetic Minority Over Sampling Technique

For this part we would be handling the class imbalance. In order to see how our model performs on the oversampled data using SMOTE. We will be using `SMOTE` API from imblearn to oversample the minority class i.e "Yes, signifying it will rain tomorrow".

Unlike our `Resample` method, we need to handle missing and categorical data before applying `SMOTE`.

In [55]:
// Perform imputation on the original dataset using "mean" imputation policy.
Impute("weatherAUS.csv");

In [56]:
!cat ./data/weatherAUS_mean_imputed.csv | sed 1d > ./data/weatherAUS_mean_imp.csv

In [57]:
!cut -d, -f1 --complement ./data/weatherAUS_mean_imp.csv > ./data/weatherAUS_trim2.csv

In [58]:
!rm ./data/weatherAUS_trim.csv

In [59]:
!mv ./data/weatherAUS_trim2.csv ./data/weatherAUS_trim.csv

In [60]:
// Load encoded & trimmed data into armadillo matrix.
arma::mat smote = LoadData("./data/weatherAUS_trim.csv")

In [61]:
// Save the encoded armadillo matrix for applying SMOTE.
mlpack::data::Save("./data/smote_in.csv", smote);

In [ ]:
// Oversample the minority class using SMOTE resampling strategy.
Resample("./data/smote_in.csv", "RainTomorrow", "No", "Yes", "smote", "Date", 123);

In [ ]:
!cat ./data/smote_in_smotesampled.csv | sed 1d > ./data/smote_in_smotesampled_woh.csv

In [ ]:
// Load SMOTE resampled, encoded data into armadillo matrix.
arma::mat smoteEnc = LoadData("./data/smote_in_smotesampled_woh.csv")

In [ ]:
// Save the encoded armadillo matrix for visualization.
data::Save("./data/weatherAUSEnc.csv", smoteEnc);

In [ ]:
!sed -i '1iLocation,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow' ./data/weatherAUSEnc.csv

In [ ]:
// Visualize the distribution of target classes.
CountPlot("./data/weatherAUSEnc.csv", "RainTomorrow", "", "Part-3 Distribution of target class");
auto img = xw::image_from_file("./plots/Part-3 Distribution of target class.png").finalize();
img

### Visualize Correlation

In [ ]:
// Visualize the correlation of transformed dataset.
HeatMapPlot("./data/weatherAUSEnc.csv", "coolwarm", "Part-3 Correlation Heatmap", 1);
auto img = xw::image_from_file("./plots/Part-3 Correlation Heatmap.png").finalize();
img

In [ ]:
// Split the data into features (X) and target (y) variables, targets are the last row.
arma::Row<size_t> targets = arma::conv_to<arma::Row<size_t>>::from(smoteEnc.row(smoteEnc.n_rows - 1));
// Targets are dropped from the loaded matrix.
smoteEnc.shed_row(smoteEnc.n_rows-1);

### Train Test Split
The dataset has to be split into training and test set. The test ratio is taken as 25% of the total observations.

In [ ]:
// Split the dataset into train and set sets using mlpack Split API.
Split(smoteEnc, targets, Xtrain, Xtest, Ytrain, Ytest, 0.25);

In [ ]:
// Standardize the train & test features.
StandardScaler scale;
scale.Fit(Xtrain);
scale.Transform(Xtrain, XtrainScaled);
scale.Transform(Xtest, XtestScaled);

### Training the Random Forest model
We will use `RandomForest` from mlpack to train the model with 10 trees on the oversampled data.
We use only 5 trees for the sake of preserving computational resources, but better models could be obtained with more trees.

In [ ]:
// Create and train Random Forest model with 5 trees.
rf = RandomForest<>(XtrainScaled, Ytrain, 2, 5);

### Making Predictions on Test set

In [ ]:
// Predict the values for test data using previously trained model as input.
rf.Classify(XtestScaled, output, probs);

In [ ]:
// Save predicted probabilities and ground truth as csv for generating ROC AUC curve.
data::Save("./data/probabilities.csv", probs);
data::Save("./data/ytest.csv", Ytest);

### Model Evaluation

In [ ]:
// Classification report.
std::cout << "Accuracy: " << ComputeAccuracy(output, Ytest) << std::endl;
ClassificationReport(output, Ytest);

In [ ]:
// Plot ROC AUC Curve to visualize the performance of the model on TP & FP.
RocAucPlot("./data/ytest.csv", "./data/probabilities.csv", "Part-3 SMOTE ROC AUC Curve");
auto img = xw::image_from_file("./plots/Part-3 SMOTE ROC AUC Curve.png").finalize();
img

### Part - 4 Modelling using Random undersampling
For this part we would be handling the class imbalance by undersampling the majority class, to see how well our model trains and performs on randomly undersampled data.

In [ ]:
// Undersample the minority population.
Resample("weatherAUS.csv", "RainTomorrow", "No", "Yes", "undersample", "Date", 123);

In [ ]:
// Visualize the distribution of target classes.
CountPlot("./data/weatherAUS_undersampled.csv", "RainTomorrow", "", "Part-4 Undersampled Population");
auto img = xw::image_from_file("./plots/Part-4 Undersampled Population.png").finalize();
img

In [ ]:
// Generate a heatmap for missing values.
MissingPlot("./data/weatherAUS_undersampled.csv", "PuBu", "Part-4 Missing values pre-imputation");
auto img = xw::image_from_file("./plots/Part-4 Missing values pre-imputation.png").finalize();
img

In [ ]:
// Imputation using mean.
Impute("./data/weatherAUS_undersampled.csv");

In [ ]:
!cat ./data/weatherAUS_undersampled_mean_imputed.csv | sed 1d > ./data/weatherAUS_us_imp.csv

In [ ]:
!cut -d, -f1 --complement ./data/weatherAUS_us_imp.csv > ./data/weatherAUS_trim2.csv

In [ ]:
!rm ./data/weatherAUS_trim.csv

In [ ]:
!mv ./data/weatherAUS_trim2.csv ./data/weatherAUS_trim.csv

In [ ]:
// Load undersampled encoded & trimmed data into armadillo matrix.
arma::mat underSampled = LoadData("./data/weatherAUS_trim.csv")

In [ ]:
// Save the armadillo matrix for visualization.
data::Save("./data/weatherAUSEnc.csv", underSampled);

In [ ]:
!sed -i '1iLocation,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow' ./data/weatherAUSEnc.csv

In [ ]:
// Visualize the correlation of transformed dataset.
HeatMapPlot("./data/weatherAUSEnc.csv", "coolwarm", "Part-4 Correlation Heatmap", 1);
auto img = xw::image_from_file("./plots/Part-4 Correlation Heatmap.png").finalize();
img

In [ ]:
// Split the data into features (X) and target (y) variables, targets are the last row.
arma::Row<size_t> targets = arma::conv_to<arma::Row<size_t>>::from(underSampled.row(underSampled.n_rows - 1));
// Targets are dropped from the loaded matrix.
underSampled.shed_row(underSampled.n_rows-1);

### Train Test Split
The dataset has to be split into training and test set. Here the dataset has 63754 observations and the test ratio is taken as 25% of the total observations. This indicates that the test set should have 25% * 63754 = 15938 observations and training set should have 47816 observations respectively.

In [ ]:
// Split the dataset into train and set sets using mlpack Split API.
Split(underSampled, targets, Xtrain, Xtest, Ytrain, Ytest, 0.25);

In [ ]:
// Standardize the train & test features.
StandardScaler scale;
scale.Fit(Xtrain);
scale.Transform(Xtrain, XtrainScaled);
scale.Transform(Xtest, XtestScaled);

### Training the Random Forest model
We will use `RandomForest` from mlpack to train the model with 10 trees on the oversampled data.
We use only 5 trees for the sake of preserving computational resources, but better models could be obtained with more trees.

In [ ]:
// Create and train Random Forest model with 5 trees.
rf = RandomForest<>(XtrainScaled, Ytrain, 2, 5);

### Making Predictions on Test set

In [ ]:
// Predict the values for test data using previously trained model as input.
rf.Classify(XtestScaled, output, probs);

In [ ]:
// Save predicted probabilities and ground truth as csv for generating ROC AUC curve.
data::Save("./data/probabilities.csv", probs);
data::Save("./data/ytest.csv", Ytest);

### Model Evaluation

In [ ]:
// Classification report.
std::cout << "Accuracy: " << ComputeAccuracy(output, Ytest) << std::endl;
ClassificationReport(output, Ytest);

In [ ]:
// Plot ROC AUC Curve to visualize the performance of the model on TP & FP.
RocAucPlot("./data/ytest.csv", "./data/probabilities.csv", "Part-4 Random Undersampled targets ROC AUC Curve");
auto img = xw::image_from_file("./plots/Part-4 Random Undersampled targets ROC AUC Curve.png").finalize();
img

From the above classification report, we can infer that our model trained on undersampled data performs well on both the classes compared to imbalanced model in Part 1. Also from the ROC AUC Curve, we can infer the True Positive Rate is around 80%, which is better than imbalanced model, but still performs worse than oversampled model.

### Conclusion
Both random oversampled & SMOTE model performs well on test data, random undersampled model performs better compared to imbalanced model, but there is still room for improvement. Feel free to play around with the hyperparameters, training data split ratio etc. 